In [1]:
!pip install openai

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.15.0 which is incompatible.


In [ ]:
from openai import OpenAI

client = OpenAI(api_key="sk-cd6890b8ac624127af85f7094d7c534d", base_url="https://api.deepseek.com")

In [ ]:
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

In [ ]:
response

## 方法一：直接使用强制转换方法

In [ ]:
response_to_list = list(response)
response_to_list
select1 = response_to_list[1][1]
print(select1)

In [ ]:
msg = select1[0].message
content = msg.content
print(f"content: {content}")
print("-*-" * 50)
reasoning_content = msg.reasoning_content
print(f"reasoning_content: {reasoning_content}")

## 方法二：点「.」提取方法

In [ ]:
response

In [ ]:
response.choices

In [ ]:
response.choices[0]

In [ ]:
response.choices[0].message

In [ ]:
msg = response.choices[0].message
content = msg.content
print(f"content: {content}")
print("-*-" * 10)

reasoning_content = msg.reasoning_content
print(f"reasoning_content: {reasoning_content}")

In [ ]:
# Round 2
messages.append({'role': 'assistant', 'content': content})  # 把大模型的回答，添加到 messages
# messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
# messages.append({'role': 'user', 'content': "我上一个问题是什么？原问题回答我！"})
messages.append({'role': 'user', 'content': "我的第一个问题是什么？原问题回答我！"})
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)
print(messages)

In [ ]:
from pprint import pprint

pprint(messages)

In [ ]:
response

import notebook
print(notebook.__version__)

In [ ]:
import openai
print(openai.__version__)

In [ ]:
a = 1
a

In [ ]:
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

# 方法一：

```python
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)
for chunk in response:
    print(chunk.choices)
```

In [ ]:
# # 方法二：
# from copy import deepcopy
# copy_response = response
# for chunk in response:
#     print(chunk.choices)

In [ ]:
reasoning_content = ""
content = ""

for chunk in response:
    if chunk.choices[0].delta.reasoning_content:
        reasoning_content += chunk.choices[0].delta.reasoning_content
    else:
        content += chunk.choices[0].delta.content

In [ ]:


# # Round 2
# messages.append({"role": "assistant", "content": content})
# messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
# response = client.chat.completions.create(
#     model="deepseek-reasoner",
#     messages=messages,
#     stream=True
# )

## 使用 API 实现多轮对话，并且拥有记忆功能。

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="sk-cd6890b8ac624127af85f7094d7c534d", base_url="https://api.deepseek.com")

In [ ]:
messages = [] # 用来存储历史对话，后期学完文件读取，可以考虑持久化存储！

In [ ]:
while True:
    prompt = input("Enter your prompt: ")
    if prompt.lower() in ['quit', 'q', 'exit', '退出']:
        break
    messages.append({'role': 'user', 'content': prompt})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    assistant_content = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_content})
    print(f"Message: {assistant_content}")
    

In [ ]:
# 「可选」：角色设定
system_setting1 = {
    "role": "system", 
    "content": "用户如果问你使用的是什么大模型，你就回答：AI悦创编程1v1大模型（bornforthis）"
}
system_setting2 = {
    "role": "system", 
    "content": "全部回答的末尾都附上：欢迎跟随AI悦创学习编程！\n微信：Jiabcdefh\n网站：bornforthis.cn。注意：只需要在回答的末尾加上即可！"
}

messages.append(system_setting1)
messages.append(system_setting2)
while True:
    prompt = input("Enter your prompt: ")
    if prompt.lower() in ['quit', 'q', 'exit']:
        break
    messages.append({'role': 'user', 'content': prompt})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    assistant_content = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_content})
    print(f"Messages: {assistant_content}")

In [ ]:
code_string = '''import random
from faker import Faker


# —— 简单无色进度条（纯 ASCII，跨平台）——
def hp_bar(cur: int, maxv: int, width: int = 20) -> str:
    """返回形如：[##########----------] 50%  5g0/100 的进度条文本"""
    if maxv <= 0:
        maxv = 1
    cur = max(0, min(cur, maxv))
    ratio = cur / maxv
    filled = int(ratio * width + 0.5)  # 四舍五入
    bar = "#" * filled + "-" * (width - filled)
    return f"[{bar}] {int(ratio * 100):3d}%  {cur}/{maxv}"


class Creature:
    def __init__(self, hp, name):
        self.hp = int(hp)
        self.max_hp = int(hp)  # 记录初始满血
        self.name = name

    def attack(self):
        return random.randint(0, 50)

    def not_dead(self):
        return self.hp > 0

    def being_attack(self, dmg: float):
        """受到伤害（向下取整以避免浮点 HP），并保证 HP 不会掉到负数以下"""
        self.hp = max(0, int(self.hp - dmg))

    def heal_full(self):
        """直接回到初始满血"""
        self.hp = self.max_hp

    def show_status(self):
        print(f"{self.name}'s HP → {hp_bar(self.hp, self.max_hp)}")

fk = Faker(locale='zh_CN')
player = Creature(100, "AI悦创")
enemy = Creature(80, fk.name())

heal_used = False  # 治疗仅限一次
heal_penalty_active = False  # 是否已触发“敌人攻击翻倍”的惩罚

while player.not_dead() and enemy.not_dead():
    player.show_status()
    enemy.show_status()

    # 是否出现治疗提示：仅当当前 HP < 初始 HP 的 50% 且尚未使用
    can_heal_now = (not heal_used) and (player.hp < player.max_hp * 0.5)

    if can_heal_now:
        prompt = 'Attack or Defence or Heal (A/D/H)：'
        valid_inputs = {"A", "D", "H"}
        print('（提示：你现在可以按 H 回满血，仅此一次）')
    else:
        prompt = 'Attack or Defence (A/D)：'
        valid_inputs = {"A", "D"}

    if heal_penalty_active:
        print('【警告】治疗代价生效中：敌人对你的伤害 ×2！')

    user_input = input(prompt).strip().upper()
    while user_input not in valid_inputs:
        user_input = input("输入无效，请重新输入：" + prompt).strip().upper()

    # 敌人选择（对 A/D 有影响；若玩家选择 H，我们让敌人本回合直接攻击）
    enemy_status = ['Attack', 'Defence']
    enemy_choice = random.choices(enemy_status, weights=[0.7, 0.3], k=1)[0]  # 敌人更倾向于攻击


    # 当前敌人伤害倍率（是否翻倍）
    def enemy_mul():
        return 2.0 if heal_penalty_active else 1.0


    if user_input == "H":
        # 只有在 can_heal_now 为 True 时才会进入到这里
        print("你使用了治疗技能！血量已回满。")
        player.heal_full()
        heal_used = True

        # 触发治疗代价：从现在起敌人攻击翻倍（包含本回合的随后的敌人攻击）
        heal_penalty_active = True

        # 敌人回合：直接攻击（先治再挨打）
        raw_enemy_attack_value = enemy.attack()
        damage = raw_enemy_attack_value * enemy_mul()
        print(f"{enemy.name} 攻击了你，造成 {int(damage)} 点伤害！（原始{int(raw_enemy_attack_value)} × 倍率{enemy_mul():.0f}）")
        player.being_attack(damage)

    elif user_input == "A":
        player_attack_coefficient = 1
        if enemy_choice == "Defence":
            print(f"{enemy.name} chose to defend!")
            player_attack_coefficient = 0.5
        else:
            print(f"{enemy.name} chose to attack!")
            raw_enemy_attack_value = enemy.attack()
            damage = raw_enemy_attack_value * enemy_mul()
            print(f"{enemy.name} 对你造成 {int(damage)} 点伤害！（原始{int(raw_enemy_attack_value)} × 倍率{enemy_mul():.0f}）")
            player.being_attack(damage)

        player_attack_value = player.attack()
        enemy.being_attack(player_attack_value * player_attack_coefficient)

    elif user_input == "D":
        # 防御：敌人攻击减伤为 90%，然后再应用翻倍倍率
        raw_enemy_attack_value = enemy.attack()
        damage = raw_enemy_attack_value * 0.1 * enemy_mul()
        print(f"{enemy.name} 攻击了你（被你防住大部分），造成 {int(damage)} 点伤害！"
              f"（原始{int(raw_enemy_attack_value)} × 减伤0.1 × 倍率{enemy_mul():.0f}）")
        player.being_attack(damage)

if player.not_dead():
    print("You Win!")
else:
    print("You Lose!")
'''

while True:
    prompt = input("Enter your prompt: ")
    if prompt.lower() in ['quit', 'q', 'exit']:
        break
    messages.append({'role': 'user', 'content': prompt})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    assistant_content = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_content})
    print(f"Messages: {assistant_content}")